In [1]:
import sqlite3
import pandas as pd, numpy as np
from demjson import decode
from datetime import datetime
import os
import pandas.io.sql as sql

In [2]:
def create_conn(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return None

In [3]:
conn = create_conn('mts_.db')
conn.execute("PRAGMA foreign_keys = 1") # enable foreign keys
cursor = conn.cursor()

In [4]:
def create_table(conn, statement):
    try:
        c = conn.cursor()
        c.execute(statement)
    except Error as e:
        print(e)

In [5]:
assert 1 == list(conn.execute('pragma foreign_keys'))[0][0] #test enabled foreign keys

In [6]:
create_table(conn, """
CREATE TABLE IF NOT EXISTS users (
 id integer NOT NULL PRIMARY KEY,
 name text NULL,
 created_at datetime
);
""")

create_table(conn, """
create table if not exists country (
country_code varchar(10) NOT NULL PRIMARY KEY,
country varchar(255) NOT NULL
);
""")

create_table(conn, """
create table if not exists tweets (
id integer NOT NULL PRIMARY KEY,
user_id integer NULL,
tweet_text varchar(255),
url text,
lang varchar(10),
loc varchar(255),
coordinates text,
country_code varchar(10) NULL,
created_at datetime,
        FOREIGN KEY (user_id) REFERENCES users(id) on delete cascade,
        FOREIGN KEY (country_code) REFERENCES country(country_code) on delete cascade
);
""")

create_table(conn, """
create table if not exists tweet_sentiment (
id integer not null primary key,
sentiment real,
    FOREIGN KEY (id) REFERENCES tweets(id) on delete cascade
);
""")          

In [7]:
sent=open('AFINN-111.txt', 'r')
SentDict={}
err_lines=[]
for idx, row in enumerate(sent):
    try:
        r = row.split('\t')
        SentDict[r[0]]= float(r[1])
    except BaseException:
        err_lines.append(idx)
        
def text_sentiment(text):
    try:
        text = str(text)
    except BaseException:
        return 0
    text_list = text.split(' ')
    vals = [SentDict.get(i) for i in text_list if SentDict.get(i) is not None]
    return np.mean(vals) if len(vals)>0 else 0
        

In [8]:
f = open("three_minutes_tweets.json.txt", "r")
jsonarr=[]
for i in f:
    a = decode(i)
    jsonarr.append(a)

In [9]:
df = pd.DataFrame.from_dict(jsonarr, orient='columns')
#df = df[~pd.isnull(df.id)]

In [10]:
def extract(x, foo):
    try:
        return x[foo]
    except BaseException: 
        return None

In [11]:
def todt(timestamp_ms):
    """converts ms time into YYYY-MM-DD HH:MM:SS"""
    try:
        return datetime.fromtimestamp(float(int(timestamp_ms)*0.001))
    except BaseException:
        return None

In [12]:
#df['user_lang']= df['user'].apply(lambda x: extract(x, 'lang'))
df['user_created_at'] = df['user'].apply(lambda x: extract(x, 'created_at'))
df['created_at'] = df['timestamp_ms'].apply(lambda x: todt(x) )
df['name'] = df['user'].apply(lambda x: extract(x, 'screen_name'))
df['user_id'] = df['user'].apply(lambda x: extract(x, 'id_str'))
df['loc'] = df['user'].apply(lambda x: extract(x, 'location'))
df['coordinates_polygon'] = df['place'].apply(lambda x : extract(extract(x, 'bounding_box'), 'coordinates'))
df['tweet_id']=df.id_str


In [13]:
df['country_code'] = df['place'].apply(lambda x: extract(x, 'country_code'))
df['country'] = df['place'].apply(lambda x: extract(x, 'country'))
df['display_url']=df[['name', 'tweet_id']].apply(lambda r: f'http://twitter.com/{r[0]}/status/{r[1]}' , axis = 1)


In [14]:
users = df[['user_id','name', 'user_created_at']].drop_duplicates(subset = ['user_id', 'user_created_at', 'name'])

In [15]:
users.columns = ['id', 'name', 'created_at']
assert len(users)==len(users.id.unique())
users = users[~pd.isnull(users.id)]

In [16]:
users.to_sql('users', con=conn, if_exists='append', index=False)

In [17]:
country = df[['country_code', 'country']]

In [18]:
country = df[~pd.isnull(df.country_code)][['country_code', 'country']].drop_duplicates(subset='country_code')
assert len(country)==len(country.country_code.unique())
country = country[~pd.isnull(country.country_code)]

In [19]:
country.to_sql("country", conn, if_exists="append", index=False)

In [20]:
tweets = df[['tweet_id', 'user_id', 'text', 'display_url', 
             'lang', 'loc', 'coordinates_polygon', 
             'country_code', 'created_at']]
tweets.columns = ['id', 'user_id' ,'tweet_text', 'url', 'lang', 'loc', 'coordinates', 'country_code', 'created_at']
def smartstr(x):
    try:
        return str(x)
    except BaseException:
        return None
tweets.coordinates = tweets.coordinates.apply(lambda x: smartstr(x))

/home/sedovda/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [22]:
tweets=tweets.sort_values(by='created_at', ascending=False).drop_duplicates(subset = 'id', keep='first')
tweets = tweets[~pd.isnull(tweets.id)]

In [23]:
tweets.to_sql("tweets", conn, if_exists="append", index=False)

In [24]:
df['sentiment'] = df.text.apply(lambda x: text_sentiment(x))

In [25]:
tweet_sentiment = df[['tweet_id', 'sentiment']]
tweet_sentiment.columns = ['id', 'sentiment']


In [26]:
tweet_sentiment = tweet_sentiment.drop_duplicates(subset = 'id', keep='first')
tweet_sentiment = tweet_sentiment[~pd.isnull(tweet_sentiment.id)]
tweet_sentiment.to_sql("tweet_sentiment", conn, if_exists="append", index=False)

In [27]:
conn.commit()

In [28]:
def open_query(query, conn = conn):
    df = sql.read_sql(query, conn)
    return df


In [29]:
# так пользователи чаще всего не имеют страну и локацию имеет смысл смотреть их без привязки к стране и локации
# страну и локацию рассмотрим отдельно

In [30]:
most_less_happiest_users = open_query("""
;with t1 as (select
u.name,
avg(s.sentiment) sentiment
, GROUP_CONCAT(tweet_text, '\n') text
    from tweets t
    inner join tweet_sentiment s on s.id = t.id
    inner join users u on u.id = t.user_id
group by 1
)
select t1.name, t1.sentiment, text as tweets from t1 
where t1.sentiment = (select max(sentiment) from t1) or t1.sentiment = (select min(sentiment) from t1)
order by sentiment desc
;""")
# получилось больше 2х строк тк одинакомые мин и макс значения имеют более 1 страны
most_less_happiest_users

,name,sentiment,tweets
0,AllyyKatttttt_,4.0,RT @Malikajanae: @AllyyKatttttt_ lmao text me
1,CAL_SMILES,4.0,RT @RyannMinajj: finally got this masterpiece ...
2,Connor_Early,4.0,Can't wait to come back home and do all these ...
3,Franke_Ray,4.0,Laugh of the day lmao http://t.co/5u28o7MfkH
4,HolzhauerAsh,4.0,I'm blessed to have a wonderful wonderful husb...
5,IngeLiseVejlgaa,4.0,@IngeLiseVejlgaa Thank you for fav and Rt **T...
6,Jasminesroom95,4.0,RT @RyannMinajj: finally got this masterpiece ...
7,JasonBrownTWCN,4.0,Pole-sitter Matt Kenseth wins at Michigan #NASCAR
8,Knighter01,4.0,RT @jim_utter: The 'Month of Kenseth' continue...
9,SenHHS,4.0,RT @carmenmmachado: Interfictions submissions ...


In [31]:
most_less_happiest_countries_and_locations = open_query("""
;with t1 as (select
c.country,
avg(s.sentiment) sentiment
, GROUP_CONCAT(tweet_text, '\n') text
    from tweets t
    inner join tweet_sentiment s on s.id = t.id
    inner join country c on c.country_code = t.country_code
group by 1
)
select t1.country, t1.sentiment, text as tweets from t1 
where t1.sentiment = (select max(sentiment) from t1) or t1.sentiment = (select min(sentiment) from t1)
order by sentiment desc
;""")
most_less_happiest_countries_and_locations

,country,sentiment,tweets
0,Danmark,4.0,@IngeLiseVejlgaa Thank you for fav and Rt **T...
1,Republika ng Pilipinas,-3.0,Ganyan talaga some will hate you at kung sino ...


In [146]:
conn.commit()
conn.close()
